In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

#Localizar o consolidado mais recente
BASE_DIR = Path("../datas")
CONSOLIDADOR_DIR = BASE_DIR / "arquivos_consolidados"

arquivos = sorted(CONSOLIDADOR_DIR.glob("consolidado_*.csv"),key=lambda p:p.stat().st_mtime, reverse=True)

if not arquivos:
    raise FileNotFoundError("Nenhum arquivo consolidado encontrado")

arquivo_consolidado = arquivos[0]
print(f"Usando o arquivo: {arquivo_consolidado.name}")

#Lendo o arquivo consolidado

df = pd.read_csv(arquivo_consolidado,sep=";",encoding="utf-8")

#PADRONIZAÇÕES ------------------------------------------------------------------------

#Renomear coluna item para evento
df.rename(columns={'item':'EVENTO'},inplace=True)



#Convertendo colunas monetárias

def to_float(x):
    try:
        return float(str(x).replace("$","").replace(".","").replace(",","."))
    except Exception:
        return 0.0
    

for col in ["valor_liquido","valor_pagamento","saldo_devedor"]:
    if df[col].dtype == object:
        df[col] = df[col].apply(to_float)


#converter datas para o datetime

for dcol in ['data_pagamento','data_liquidacao','data_vencimento']:
    if dcol in df.columns:
        df[dcol] = pd.to_datetime(df[dcol],errors="coerce")


#Metricas adicionais

df["percentual_inadimplencia"] = df.apply(lambda r: (r["saldo_devedor"] / r["valor_liquido"] * 100) if r['valor_liquido'] != 0 else 0,
                                          axis=1)


# top_inad = df.sort_values(by='percentual_inadimplencia',ascending=False).head(10)
# display(top_inad[['valor_bruto','valor_liquido','saldo_devedor','percentual_inadimplencia']])



#Resumos
# Resumo por EVENTO ________

resumo_geral = (df.groupby("EVENTO"))




Usando o arquivo: consolidado_20250417_112637.csv


,valor_bruto,valor_liquido,saldo_devedor,percentual_inadimplencia
5743,"$5.360,00",4556.0,4662.16,102.330114
1583,"$1.239,00",1239.0,1267.87,102.330105
5741,"$1.239,00",1239.0,1267.87,102.330105
1584,"$489,00",489.0,500.39,102.329243
5742,"$489,00",489.0,500.39,102.329243
1599,"$489,00",489.0,499.91,102.231084
1598,"$1.239,00",1239.0,1266.64,102.230831
1250,"$1.239,00",1239.0,1265.01,102.099274
1247,"$1.239,00",1239.0,1265.01,102.099274
1249,"$5.360,00",4020.0,4104.38,102.099005


: 